In [7]:
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import SGD
import pickle
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from imblearn.over_sampling import SMOTE

PATH = './data/'
train_load_df = pd.read_pickle(PATH + "train_df.pkl")

N = 6000
label_count = train_load_df.label.value_counts()
under_list = label_count[label_count > N].index
df = pd.DataFrame()
for i in label_count.index:
    if i in under_list:
        df = pd.concat([df, train_load_df[train_load_df.label == i].sample(n=N)])
    else:
        df = pd.concat([df, train_load_df[train_load_df.label == i]])

vocab_size = 10000
num_classes = 12
tokenizer = Tokenizer(num_words=vocab_size)

X,Y = df.sentence, df.label
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=2, shuffle=True)
x_valid, x_test, y_valid, y_test = train_test_split(x_valid, y_valid, test_size=0.2, random_state=2, shuffle=True)

tokenizer.fit_on_texts(x_train)
X_train = tokenizer.texts_to_matrix(x_train, mode='tfidf')
X_valid = tokenizer.texts_to_matrix(x_valid, mode='tfidf')
X_test = tokenizer.texts_to_matrix(x_test, mode='tfidf')

Y_train = to_categorical(y_train, num_classes)
Y_valid = to_categorical(y_valid, num_classes)
Y_test = to_categorical(y_test, num_classes)

smt = SMOTE()
X_train_smt, Y_train_smt = smt.fit_resample(X_train, Y_train)
BATCH_SIZE = 64
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)
sgd = SGD(learning_rate=.01, momentum=.9, nesterov=True)
input_shape = X_train.shape[1]

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=256, activation='elu', input_shape=[input_shape]),
    tf.keras.layers.Dense(units=128, activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=32, activation='swish'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_classes, activation='softmax'),         
])
#model = load_model('./h5/model_SMOTE.h5')
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(X_train_smt, Y_train_smt, batch_size=BATCH_SIZE, epochs=100, verbose=1, validation_split=0.1, callbacks=[early_stop])

test_loss, test_acc = model.evaluate(X_valid, Y_valid, verbose=2)
print(test_loss, test_acc)

Epoch 1/100
815/815 [==============================] - 4s 5ms/step - loss: 1.5844 - accuracy: 0.4957 - val_loss: 1.1976 - val_accuracy: 0.6325
Epoch 2/100
815/815 [==============================] - 4s 5ms/step - loss: 1.1979 - accuracy: 0.6128 - val_loss: 1.1053 - val_accuracy: 0.6528
Epoch 3/100
815/815 [==============================] - 4s 5ms/step - loss: 1.0478 - accuracy: 0.6594 - val_loss: 1.0362 - val_accuracy: 0.6895
Epoch 4/100
815/815 [==============================] - 4s 5ms/step - loss: 0.9477 - accuracy: 0.6892 - val_loss: 1.0097 - val_accuracy: 0.6983
Epoch 5/100
815/815 [==============================] - 4s 5ms/step - loss: 0.8675 - accuracy: 0.7138 - val_loss: 1.0267 - val_accuracy: 0.6971
Epoch 6/100
815/815 [==============================] - 4s 5ms/step - loss: 0.8002 - accuracy: 0.7343 - val_loss: 1.0427 - val_accuracy: 0.7062
Epoch 7/100
815/815 [==============================] - 4s 5ms/step - loss: 0.7420 - accuracy: 0.7521 - val_loss: 1.0363 - val_accuracy: 0.7149